In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_voltage_temp(dataset, title = ''):
    fig, axs = plt.subplots(4, 2, figsize=(8, 8), dpi=200)
    fig.suptitle(title)

    for i in range(1, 8):
        ax = axs[(i-1)//2, (i-1)%2]
        ax.scatter(1/dataset['T'], dataset[f'v{i}'], s=5)
        ax.set_title(f'v{i}')

    fig.supxlabel('T (1/K)')
    fig.supylabel('Voltage (V)')
    plt.tight_layout()
    plt.show() 